# Transactions

Desejamos construir uma aplicação de empréstimos peer-to-peer simples. Nesta aplicação os usuários podem:
 
- Depositar uma certa quantia em sua conta;
- Sacar da conta, desde que tenham saldo;
- Criar um "emprestimo" (mais como uma linha de credito, realmente) entre dois usuarios
- Pagar (parcial ou totalmente) um emprestimo
- Tomar mais dinheiro ainda de um emprestimo (ou linha de credito)

Mais ainda, temos o requisito de que todas as operações devem ser armazenadas no banco de dados.

Vamos agrupar esses requerimentos como segue:

- **Movimentação de conta**: depositar ou sacar da conta;
- **Criação de empréstimo**: disponibilizar uma linha de credito (sem nenhum dinheiro transferido ser necessariamente transferido);
- **Operação em empréstimo**:
    - **adicionar valor** (como quando o devedor saca da linha de credito, ou quando os juros são computados):
        - Deve gerar duas movimentacões:
            - Depósito no devedor;
            - Saque no credor;
    - **remover valor** (como quando o devedor paga alguma quantia):
        - Deve gerar duas movimentacões:
            - Saque no devedor;
            - Depósito no credor.

A modelagem relacional deste problema resultou no seguinte diagrama:

<img src='img/diagrama.png' alt='Diagrama do modelo relacional' style='width: 500px;'/>

que traduzimos no script de criação da base de dados como segue:

---
```SQL
DROP DATABASE IF EXISTS emprestimos;
CREATE DATABASE emprestimos;
USE emprestimos;

CREATE TABLE usuario (
    id_usuario INT NOT NULL AUTO_INCREMENT,
    nome VARCHAR(80) NOT NULL,
    sobrenome VARCHAR(80) NOT NULL,
    saldo DECIMAL(30 , 2 ) NOT NULL DEFAULT 0.0,
    PRIMARY KEY (id_usuario),
    CONSTRAINT c_saldo CHECK (saldo >= 0.0)
);

CREATE TABLE emprestimo (
    id_emprestimo INT NOT NULL AUTO_INCREMENT,
    id_credor INT NOT NULL,
    id_devedor INT NOT NULL,
    valor_atual DECIMAL(30 , 2 ) NOT NULL DEFAULT 0.0,
    data_inicio DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    data_modificação DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    PRIMARY KEY (id_emprestimo),
    CONSTRAINT fk_credor FOREIGN KEY (id_credor)
        REFERENCES usuario (id_usuario),
    CONSTRAINT fk_devedor FOREIGN KEY (id_devedor)
        REFERENCES usuario (id_usuario),
    CONSTRAINT c_valor CHECK (valor_atual >= 0.0)
);

CREATE TABLE operacao (
    id_operacao INT NOT NULL AUTO_INCREMENT,
    id_emprestimo INT NOT NULL,
    valor DECIMAL(30 , 2 ),
    data_operacao DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    PRIMARY KEY (id_operacao),
    CONSTRAINT fk_emprestimo FOREIGN KEY (id_emprestimo)
        REFERENCES emprestimo (id_emprestimo)
);

CREATE TABLE movimentacao (
    id_movimentacao INT NOT NULL AUTO_INCREMENT,
    id_usuario INT NOT NULL,
    valor DECIMAL(30 , 2 ),
    data_operacao DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    PRIMARY KEY (id_movimentacao),
    CONSTRAINT fk_usuario FOREIGN KEY (id_usuario)
        REFERENCES usuario (id_usuario)
);

```

---

Rode esse script (`emprestimos.sql`) para criar a base de dados no MySQL. Vamos também trazer o código de conexão à base de dados:

In [1]:
import mysql.connector
import os
from functools import partial
from dotenv import load_dotenv

In [20]:
load_dotenv(override=True)

def get_connection_helper():

    def run_db_query(connection, query, args=None):
        with connection.cursor() as cursor:
            print("Executando query:")
            for result in cursor.execute(query, multi=True):
                if result.with_rows:
                    for row in result.fetchall():
                        print(row)
                else:
                    print(f"{result.rowcount} linhas afetadas.")

    connection = mysql.connector.connect(
        host=os.getenv("MD_DB_SERVER"),
        port=int(os.getenv("MD_DB_PORT", 3306)),
        user=os.getenv("MD_DB_USERNAME"),
        password=os.getenv("MD_DB_PASSWORD"),
        database="emprestimos",
    )
    return connection, partial(run_db_query, connection)


connection, db = get_connection_helper()

## Transactions

Uma *transação* é um grupo de operações na base de dados, em uma dada *sessão*. Vamos conhecer o conceito de transação com as atividades a seguir.

### `COMMIT`

Vamos gerar alguns usuários na nossa base:

In [3]:
db("""
INSERT INTO usuario (id_usuario, nome, sobrenome) VALUES
    (1, 'Juca', 'Silva'), 
    (2, 'Mario', 'Ferreira'), 
    (3, 'Ana', 'Soares'), 
    (4, 'Antonio', 'Reis'), 
    (5, 'Paulo', 'Oliveira')
""")

Executando query:
5 linhas afetadas.


Vamos verificar se os usuários foram inseridos:

In [4]:
db("SELECT * FROM usuario")

Executando query:
(1, 'Juca', 'Silva', Decimal('0.00'))
(2, 'Mario', 'Ferreira', Decimal('0.00'))
(3, 'Ana', 'Soares', Decimal('0.00'))
(4, 'Antonio', 'Reis', Decimal('0.00'))
(5, 'Paulo', 'Oliveira', Decimal('0.00'))


Excelente, parece que deu tudo certo! Agora vamos verificar diretamente no MySQL Workbench, só para ter certeza:

```SQL
SELECT * FROM usuario;
```

<img src='img/usuarios_null.png' alt='Nenhum usuario na tabela!' style='width: 400px;'/>

Epa, cade os usuarios?

Para entender o que aconteceu, precisamos compreender melhor como funciona uma conexão com o banco de dados. Quando criamos um objeto de conexão, estamos iniciando uma *sessão* do banco de dados. Em uma sessão as várias operações sendo realizadas configuram uma *transação*. 

As transações não são efetuadas diretamente no armazenamento permanente, a não ser que o banco tenha a propriedade de *'auto-commit'*. Vamos verificar se esse é o caso na nossa conexão:

In [5]:
print(connection.autocommit)

False


Você deve ver o valor 'False' acima. Isso significa que devemos explicitamente indicar quando estamos satisfeitos com uma transação, para que os valores inseridos (ou removidos) sejam efetivamente registrados no banco de dados. Isso é feito através do comando `COMMIT`:

In [5]:
db("COMMIT")

Executando query:
0 linhas afetadas.


Agora consulte o banco de dados via MySQL Workbench (ou seja, usando uma sessão diferente da nossa):


```SQL
SELECT * FROM usuario;
```

<img src='img/usuarios_commit.png' alt='Agora tem usuario na tabela!' style='width: 400px;'/>

Pronto, agora temos usuarios no nosso banco de dados!

Outra forma de indicar o `commit` da transação é usar o método `commit()` do objeto `connection`:

```python
connection.commit()
```

### Atividade

Crie os usuários 'Carla Nakamura' e 'Maria Fontana'.

In [6]:
db("""
INSERT INTO usuario (id_usuario, nome, sobrenome) VALUES
    (6, 'Carla', 'Nakamura'), 
    (7, 'Maria', 'Fontana')
""")

Executando query:
2 linhas afetadas.


In [7]:
db("COMMIT")

Executando query:
0 linhas afetadas.


### ROLLBACK

Para efeito de testes, vamos inserir mais um usuario:

In [9]:
db("INSERT INTO usuario (nome, sobrenome) VALUES ('Fernando', 'Maia')")

Executando query:
1 linhas afetadas.


Verificando o resultado da sessão, temos:

In [10]:
db("SELECT * FROM usuario")

Executando query:
(1, 'Juca', 'Silva', Decimal('0.00'))
(2, 'Mario', 'Ferreira', Decimal('0.00'))
(3, 'Ana', 'Soares', Decimal('0.00'))
(4, 'Antonio', 'Reis', Decimal('0.00'))
(5, 'Paulo', 'Oliveira', Decimal('0.00'))
(6, 'Carla', 'Nakamura', Decimal('0.00'))
(7, 'Maria', 'Fontana', Decimal('0.00'))
(8, 'Fernando', 'Maia', Decimal('0.00'))


Agora, se executarmos o `COMMIT` a nossa modificação se tornará permanente. Porém suponha que estamos arrependidos, e não queremos prosseguir com o `COMMIT` da transação, o que fazer?

Para abandonar as mudanças da transação atual, usamos o comando `ROLLBACK`:

In [11]:
db("ROLLBACK")

Executando query:
0 linhas afetadas.


Verificando o estado do banco de dados na sessão atual temos:

In [12]:
db("SELECT * FROM usuario")

Executando query:
(1, 'Juca', 'Silva', Decimal('0.00'))
(2, 'Mario', 'Ferreira', Decimal('0.00'))
(3, 'Ana', 'Soares', Decimal('0.00'))
(4, 'Antonio', 'Reis', Decimal('0.00'))
(5, 'Paulo', 'Oliveira', Decimal('0.00'))
(6, 'Carla', 'Nakamura', Decimal('0.00'))
(7, 'Maria', 'Fontana', Decimal('0.00'))


Observe que o *rollback* só volta até o último *commit*, e a repetição de *rollbacks* NÃO volta mais ainda para o passado! `COMMIT` e `ROLLBACK` são definitivos!

O *rollback* pode ser obtido também com o método `rollback()` do objeto `connection`:

```Python
connection.rollback()
```

### Atividade

Insira o usuario 'Walter Branco' mas não execute um `COMMIT`, e sim um `ROLLBACK`. Verifique que o `ROLLBACK` funcionou.

In [13]:
db("INSERT INTO usuario (nome, sobrenome) VALUES ('Walter', 'Branco')")

Executando query:
1 linhas afetadas.


Verificando o resultado da sessão, temos:

In [14]:
db("SELECT * FROM usuario")

Executando query:
(1, 'Juca', 'Silva', Decimal('0.00'))
(2, 'Mario', 'Ferreira', Decimal('0.00'))
(3, 'Ana', 'Soares', Decimal('0.00'))
(4, 'Antonio', 'Reis', Decimal('0.00'))
(5, 'Paulo', 'Oliveira', Decimal('0.00'))
(6, 'Carla', 'Nakamura', Decimal('0.00'))
(7, 'Maria', 'Fontana', Decimal('0.00'))
(9, 'Walter', 'Branco', Decimal('0.00'))


Fazemos o `rollback`:

In [15]:
db("ROLLBACK")

Executando query:
0 linhas afetadas.


Observe que o usuário 'Walter Branco' não é mais retornado:

In [16]:
db("SELECT * FROM usuario")

Executando query:
(1, 'Juca', 'Silva', Decimal('0.00'))
(2, 'Mario', 'Ferreira', Decimal('0.00'))
(3, 'Ana', 'Soares', Decimal('0.00'))
(4, 'Antonio', 'Reis', Decimal('0.00'))
(5, 'Paulo', 'Oliveira', Decimal('0.00'))
(6, 'Carla', 'Nakamura', Decimal('0.00'))
(7, 'Maria', 'Fontana', Decimal('0.00'))


### `START TRANSACTION`

Para garantir que o banco de dados MySQL não está operando em modo *auto-commit* use o comando `START TRANSACTION`. Este comando garante que estamos iniciando uma transação sem *auto-commit*. Por exemplo:

In [8]:
db("START TRANSACTION")
db("INSERT INTO usuario (nome, sobrenome) VALUES ('Chapeleiro', 'Maluco')")
db("COMMIT")
db("SELECT * FROM usuario")

Executando query:
0 linhas afetadas.
Executando query:
1 linhas afetadas.
Executando query:
0 linhas afetadas.
Executando query:
(1, 'Juca', 'Silva', Decimal('0.00'))
(2, 'Mario', 'Ferreira', Decimal('0.00'))
(3, 'Ana', 'Soares', Decimal('0.00'))
(4, 'Antonio', 'Reis', Decimal('0.00'))
(5, 'Paulo', 'Oliveira', Decimal('0.00'))
(6, 'Carla', 'Nakamura', Decimal('0.00'))
(7, 'Maria', 'Fontana', Decimal('0.00'))
(8, 'Chapeleiro', 'Maluco', Decimal('0.00'))


Vamos reverter essa inserção em uma outra transação.

In [9]:
db("START TRANSACTION")
db("""
    SELECT id_usuario INTO @id_usuario 
    FROM usuario 
    WHERE nome = 'Chapeleiro' AND sobrenome = 'Maluco'
    ORDER BY id_usuario DESC LIMIT 1;
""")
db("DELETE FROM usuario WHERE id_usuario=@id_usuario")
db("COMMIT")
db("SELECT * FROM usuario")

Executando query:
0 linhas afetadas.
Executando query:
1 linhas afetadas.
Executando query:
1 linhas afetadas.
Executando query:
0 linhas afetadas.
Executando query:
(1, 'Juca', 'Silva', Decimal('0.00'))
(2, 'Mario', 'Ferreira', Decimal('0.00'))
(3, 'Ana', 'Soares', Decimal('0.00'))
(4, 'Antonio', 'Reis', Decimal('0.00'))
(5, 'Paulo', 'Oliveira', Decimal('0.00'))
(6, 'Carla', 'Nakamura', Decimal('0.00'))
(7, 'Maria', 'Fontana', Decimal('0.00'))


### Usando `COMMIT` e `ROLLBACK` para garantir atomicidade de transações

Vamos agora juntar os vários conceitos em um código Python para adicionar itens na tabela `emprestimo`. 

Como podemos ver no modelo E-R devemos informar os ids do credor e do devedor ao inserir uma nova linha nesta tabela, e estes ids devem ser ids válidos de clientes do nosso sistema. Logo, 
- se tentarmos inserir um emprestimo entre os usuarios 1 e 2 devemos ter sucesso, 
- mas se tentarmos inserir um emprestimo entre os usuarios 1 e 1000 devemos ter uma falha. 

Se ambos os comandos de inserção devem ser executados de modo 'tudo ou nada' (ou executam ambos, ou nenhum), parece que temos um caso ideal para aplicar transações aqui. A idéia é que se tudo acontecer sem problemas devemos executar um `COMMIT`, mas se algo der errado devemos excutar um `ROLLBACK`. Isso parece uma missão para `try`/`catch`!

In [19]:
try:
    db("START TRANSACTION")
    db("INSERT INTO emprestimo (id_credor, id_devedor) VALUES (1, 2)")
    db("INSERT INTO emprestimo (id_credor, id_devedor) VALUES (1, 1000)")
    db("COMMIT")
except Exception as e:
    print(e)
    db("ROLLBACK")

Executando query:
0 linhas afetadas.
Executando query:
1 linhas afetadas.
Executando query:
1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`emprestimos`.`emprestimo`, CONSTRAINT `fk_devedor` FOREIGN KEY (`id_devedor`) REFERENCES `usuario` (`id_usuario`))
Executando query:
0 linhas afetadas.


In [20]:
db("SELECT * FROM emprestimo")

Executando query:


### Atividade

- Execute os depósitos abaixo:

| usuario        | valor |
|----------------|-------|
| Ana Soares     | 10000 |
| Antonio Reis   | 5000  |
| Paulo Oliveira | 20000 |

Dica: crie uma função `pega_id_usuario(connection, nome, sobrenome)` para achar o id de usuario, e uma função `deposito(connection, id_usuario, valor)` para facilitar a realização dos depósitos. Um depósito requer uma inserção na tabela de movimentações, e um update no saldo.

Leia a documentação para utilizar funções adequadas: https://dev.mysql.com/doc/connector-python/en/connector-python-api-mysqlcursor.html

In [10]:
def pega_id_usuario(connection, nome, sobrenome):
    with connection.cursor() as cursor:
        cursor.execute("""
            SELECT id_usuario 
            FROM usuario 
            WHERE nome = %s AND sobrenome = %s
            ORDER BY id_usuario DESC LIMIT 1;
        """, (nome, sobrenome))
        result = cursor.fetchone()
        return result[0] if result else None

In [23]:
#Um depósito requer uma inserção na tabela de movimentações, e um update no saldo.
def faz_deposito(connection, id_usuario, valor):
    if id_usuario is None:
        raise ValueError("Usuário não encontrado")
    
    try:
        with connection.cursor() as cursor:
            cursor.execute("START TRANSACTION;")
            cursor.execute("""
                INSERT INTO movimentacao (id_usuario, data_operacao, valor) 
                VALUES (%s, NOW(), %s);
            """, (id_usuario, valor))
            cursor.execute("""
                UPDATE usuario 
                SET saldo = saldo + %s 
                WHERE id_usuario = %s;
            """, (valor, id_usuario))
            cursor.execute("COMMIT;")
    except Exception as e:
        print(e)
        cursor.execute("ROLLBACK;")

- Crie funções para consulta de saldo de conta e de valor atual de emprestimo

In [ ]:
def consulta_saldo(connection, id_usuario):
    with connection.cursor() as cursor:
        cursor.execute("""
            SELECT saldo 
            FROM usuario 
            WHERE id_usuario = %s;
        """, (id_usuario,))
        result = cursor.fetchone()
        saldo = result[0] if result else None
        print(f"Saldo do usuário {id_usuario}: {saldo}")
        return saldo

In [13]:
def valor_atual_emprestimo(connection, id_emprestimo):
    with connection.cursor() as cursor:
        cursor.execute("""
            SELECT valor_atual 
            FROM emprestimo 
            WHERE id_emprestimo = %s;
        """, (id_emprestimo,))
        result = cursor.fetchone()
        valor_atual = result[0] if result else None
        print(f"Valor atual do empréstimo {id_emprestimo}: {valor_atual}")
        return valor_atual

<span style="color:darkred; font-weight:bold">ATENÇÃO:</span> Confira, nas funções que você criou, se realizou o uso de **F-string** (ou **concatenação**) passagem de argumentos. Caso sim, refatore para utilizar `%s` e passar como `tupla` na chamada do `execute`.

- Crie os seguintes contratos de emprestimo:


| Credor | Devedor | valor |
|--|--|--|
| Ana Soares | Juca Silva | 1000 |
| Ana Soares | Antonio Reis | 2000 |
| Paulo Oliveira | Juca Silva | 3000 |

Não se esqueça que um contrato de emprestimo requer:
- criação do emprestimo
- operação de emprestimo inicial
    - operação de saque da conta do credor
    - operacao de deposito na conta do devedor
    
Use o comando `SELECT LAST_INSERT_ID()` para pegar o id do último item inserido.

Dica:
- Crie uma função para criar um empréstimo de valor zero, que retorne o id do emprestimo.
- Crie uma função para realizar um saque. Se não for possível fazer o saque, lance uma exceção.
- Crie uma função para efetuar uma operação em um empréstimo dado - isso inclui fazer o saque e o depósito também.

In [14]:
def cria_emprestimo(connection, id_credor, id_devedor):
    with connection.cursor() as cursor:
        cursor.execute("""
            INSERT INTO emprestimo (id_credor, id_devedor, valor_atual, data_inicio, data_modificação)
            VALUES (%s, %s, %s, NOW(), NOW());
        """, (id_credor, id_devedor, 0))
        connection.commit()
        # retorna id do empréstimo criado
        return cursor.lastrowid


In [15]:
def faz_saque(connection, id_usuario, valor):
    if id_usuario is None:
        raise ValueError("Usuário não encontrado")
    
    try:
        with connection.cursor() as cursor:
            cursor.execute("START TRANSACTION;")
            cursor.execute("""
                INSERT INTO movimentacao (id_usuario, data_operacao, valor) 
                VALUES (%s, NOW(), %s);
            """, (id_usuario, -valor))
            cursor.execute("""
                UPDATE usuario 
                SET saldo = saldo - %s 
                WHERE id_usuario = %s;
            """, (valor, id_usuario))
            cursor.execute("COMMIT;")
    except Exception as e:
        print(e)
        cursor.execute("ROLLBACK;")

In [16]:
#- Crie uma função para efetuar uma operação em um empréstimo dado - isso inclui fazer o saque e o depósito também.
def opera_emprestimo(connection, id_emprestimo, id_credor, id_devedor, valor):
    try:
        with connection.cursor() as cursor:
            cursor.execute("START TRANSACTION;")
            # Credor empresta dinheiro ao devedor
            cursor.execute("""
                UPDATE emprestimo 
                SET valor_atual = valor_atual + %s, data_modificação = NOW() 
                WHERE id_emprestimo = %s;
            """, (valor, id_emprestimo))
            faz_saque(connection, id_credor, valor)
            faz_deposito(connection, id_devedor, valor)
            cursor.execute("COMMIT;")
    except Exception as e:
        print(e)
        cursor.execute("ROLLBACK;")

In [17]:
id_ana_soares = pega_id_usuario(connection, 'Ana', 'Soares')
id_juca_silva = pega_id_usuario(connection, 'Juca', 'Silva')
id_antonio_reis = pega_id_usuario(connection, 'Antonio', 'Reis')
id_paulo_oliveira = pega_id_usuario(connection, 'Paulo', 'Oliveira')

print(f"id_ana_soares: {id_ana_soares}")
print(f"id_juca_silva: {id_juca_silva}")
print(f"id_antonio_reis: {id_antonio_reis}")
print(f"id_paulo_oliveira: {id_paulo_oliveira}")

id_ana_soares: 3
id_juca_silva: 1
id_antonio_reis: 4
id_paulo_oliveira: 5


In [24]:
faz_deposito(connection, id_juca_silva, 10000)
faz_deposito(connection, id_ana_soares, 10000)
faz_deposito(connection, id_antonio_reis, 10000)
faz_deposito(connection, id_paulo_oliveira, 10000)

In [26]:
db("DELETE FROM emprestimo;")
id_emprestimo_1 = cria_emprestimo(connection, id_ana_soares, id_juca_silva)
id_emprestimo_2 = cria_emprestimo(connection, id_ana_soares, id_antonio_reis)
id_emprestimo_3 = cria_emprestimo(connection, id_paulo_oliveira, id_juca_silva)
opera_emprestimo(connection, id_emprestimo_1, id_ana_soares, id_juca_silva, 1000)
opera_emprestimo(connection, id_emprestimo_2, id_ana_soares, id_antonio_reis, 2000)
opera_emprestimo(connection, id_emprestimo_3, id_paulo_oliveira, id_juca_silva, 3000)

print("Saldos finais:")
consulta_saldo(connection, id_ana_soares)
consulta_saldo(connection, id_juca_silva)
consulta_saldo(connection, id_antonio_reis)
consulta_saldo(connection, id_paulo_oliveira)

Executando query:
3 linhas afetadas.
Saldos finais:
Saldo do usuário 3: 13000.00
Saldo do usuário 1: 28000.00
Saldo do usuário 4: 24000.00
Saldo do usuário 5: 14000.00


Decimal('14000.00')

Para complementar o conhecimento, leia sobre **savepoints** em:
- https://dev.mysql.com/doc/refman/8.4/en/savepoint.html
- https://www.tutorialspoint.com/mysql/mysql_savepoint.htm

Por hoje é só!

In [27]:
connection.close()